In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv("../data/train.csv")

In [4]:
interactions = (
    train.groupby(["msno", "song_id"])
         .size()
         .reset_index(name="play_count")
)

In [5]:
user_codes = interactions["msno"].astype("category").cat.codes
item_codes = interactions["song_id"].astype("category").cat.codes

interactions["user_id"] = user_codes
interactions["item_id"] = item_codes
